Author: Lai Khee Jiunn
  
  

Source: 

    [1] http://www.bursamalaysia.com/market/listed-companies/list-of-companies/main-market/
    [2] https://klse.i3investor.com/
    [3] https://www.klsescreener.com/
  
  

Reference:

    [1] K. M. Ho, "How to make money from your stock investment even in a falling market", 3rd ed., Malaysia, Kanyin Publications Sdn. Bhd., 2017.

Note: The entire process in this jupyter notebook may take up to 2.5 hours depending on your internet speed

In [1]:
import numpy as np, pandas as pd, re
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import time
from tqdm import tqdm

### Get all listed company names and stock codes from Bursa Malaysia

In [2]:
driver = webdriver.Chrome()
driver.switch_to.window(driver.current_window_handle)
driver.maximize_window()
driver.get('http://www.bursamalaysia.com/market/listed-companies/list-of-companies/main-market/')
code, name = [],[]
x = driver.find_element_by_tag_name('table').find_element_by_tag_name('tbody').find_elements_by_tag_name('a')
for i in tqdm(x):
    y = i.get_attribute('href')
    z = y.split('=')
    if len(z) > 1:
        code.append(z[-1])
        name.append(i.get_attribute('innerText'))
driver.quit()

100%|██████████████████████████████████████████████████████████████████████████████| 1596/1596 [00:35<00:00, 45.54it/s]


In [3]:
stock_code = pd.DataFrame({'code':code,'name':name})
stock_code.loc[370,'code'] = '5235SS'
stock_code.drop(index=371,inplace=True)
stock_code.reset_index(drop=True)
print(stock_code.shape)
stock_code.head(1)

(798, 2)


,code,name
0,5250,7-ELEVEN MALAYSIA HOLDINGS BERHAD


In [4]:
stock_code.to_csv('stock_code_table.csv',index=False)

### Phase 1: Filter stocks by ROI >= 10%

In [5]:
# Get general stock information from klse.i3investor.com
start_time = time()
driver = webdriver.Chrome()
driver.switch_to.window(driver.current_window_handle)
driver.maximize_window()
error, result = [],[]
for i in tqdm(stock_code.code):
    try:
        driver.get('https://klse.i3investor.com/servlets/stk/fin/'+str(i)+'.jsp?type=last10fy')
        cat = driver.find_element_by_class_name('boarAndSector').get_attribute('innerText').split(' : ')[-1]
        x = driver.find_elements_by_tag_name('table')[13].find_elements_by_tag_name('tr')
        if len(x) != 119:
            x = driver.find_elements_by_tag_name('table')[14].find_elements_by_tag_name('tr')
        year_end = x[1].find_elements_by_tag_name('th')[3].get_attribute('innerText')
        temp = x[15].find_elements_by_tag_name('td')
        net_profit = temp[3].get_attribute('innerText')
        temp = x[42].find_elements_by_tag_name('td')
        nosh = temp[3].get_attribute('innerText')
        temp = x[84].find_elements_by_tag_name('td')
        price = temp[3].get_attribute('innerText')
        result.append([i, cat, year_end, net_profit, nosh, price])
    except:
        error.append(i)
driver.quit()
print('Time taken: ',time()-start_time)

100%|████████████████████████████████████████████████████████████████████████████████| 798/798 [57:42<00:00,  5.98s/it]


Time taken:  3468.9840862751007


In [6]:
data = pd.DataFrame(result, columns=['code','category','year_end','net_profit','num_share','price'])
data = pd.merge(data,stock_code,'left')
# net_profit and num_share are in thousands
data.net_profit = data.net_profit.apply(lambda x: int(x.replace(',','')))
data.num_share = data.num_share.apply(lambda x: int(x.replace(',','')))
data.price = data.price.apply(lambda x: float(x))
data['ROI'] = np.around(data.net_profit / (data.num_share * data.price) * 100, 2)
print(data.shape)
data.to_csv('klse_investor_roi.csv',index=False)
data.head(1)

(795, 8)


,code,category,year_end,net_profit,num_share,price,name,ROI
0,5250,Consumer,31/12/18,51307,1128891,1.5,7-ELEVEN MALAYSIA HOLDINGS BERHAD,3.03


In [7]:
data[data.ROI >= 10].category.value_counts()

Industrial Products         51
Property                    37
Consumer                    32
Construction                16
Finance                     11
Transportation&Logistics     6
Energy                       5
Plantations                  4
REITS                        3
Technology                   3
Telco&Media                  2
Utilities                    1
Name: category, dtype: int64

In [8]:
data.ROI.describe()

count      795.000000
mean        -1.516843
std        383.642897
min      -2380.550000
25%         -4.210000
50%          4.560000
75%          9.160000
max      10138.630000
Name: ROI, dtype: float64

In [9]:
data1 = data.copy()
data1 = data1.loc[data1.ROI > 10]
# The commented code below helps you to filter out the sectors which you want
# wanted = ['Consumer','Energy','Finance','Health Care','Industrial Products','Plantations','Technology']
# data1 = data1[data1.category.isin(wanted)]
data1.sort_values(['category','ROI'],ascending=[True,False],inplace=True)
data1['code'] = data1['code'].astype(str)
data1.reset_index(drop=True)
print(data1.shape)
data1.head(1)

(170, 8)


,code,category,year_end,net_profit,num_share,price,name,ROI
558,5622,Construction,31/12/18,137834,69125,0.74,PIMPINAN EHSAN BERHAD,269.46


In [10]:
# Example of Carlsberg Brewery Malaysia Berhad
data[data.code=='2836']

,code,category,year_end,net_profit,num_share,price,name,ROI
110,2836,Consumer,31/12/18,277154,305748,19.68,CARLSBERG BREWERY MALAYSIA BERHAD,4.61


In [11]:
# Get stock prices in 52 weeks range from KLSE Screener, which will be used in Phase 4
start_time = time()
driver = webdriver.Chrome()
driver.switch_to.window(driver.current_window_handle)
driver.maximize_window()
error2, result2 = [],[]
for i in tqdm(data1.code):
    try:
        driver.get('https://www.klsescreener.com/v2/stocks/view/'+i)
        week52 = driver.find_element_by_class_name('stock_details').find_elements_by_tag_name('tr')[6].find_element_by_class_name('number').get_attribute('innerText').split(' - ')
        result2.append([i, float(week52[0]), float(week52[-1])])
    except:
        error.append(i)
driver.quit()
print('Time taken: ',time()-start_time)
data2 = pd.DataFrame(result2, columns=['code','52w low','52w high'])
data2['code'] = data2['code'].astype(str)
print(data2.shape)
data2.to_csv('klse_screener_52week_price.csv',index=False)
data2.head(1)

100%|████████████████████████████████████████████████████████████████████████████████| 170/170 [08:39<00:00,  2.72s/it]


Time taken:  525.6298460960388
(170, 3)


,code,52w low,52w high
0,5622,0.61,2.6


### Phase 2: Get past 10 years data

In [12]:
# Get the last 10 years of financial data from klse.i3investor.com
start_time = time()
driver = webdriver.Chrome()
driver.switch_to.window(driver.current_window_handle)
driver.maximize_window()
error3 = []
data3 = pd.DataFrame()
for i in tqdm(data2.code):
    try:
        driver.get('https://klse.i3investor.com/servlets/stk/fin/'+str(i)+'.jsp?type=last10fy')
        cat = driver.find_element_by_class_name('boarAndSector').get_attribute('innerText').split(' : ')[-1]
        x = driver.find_elements_by_tag_name('table')[13].find_elements_by_tag_name('tr')
        if len(x) != 119:
            x = driver.find_elements_by_tag_name('table')[14].find_elements_by_tag_name('tr')
        year_end = [i,cat,'Year End']+[x.get_attribute('innerText') for x in x[1].find_elements_by_tag_name('th')[3:-1]]
        net_profit = [i,cat,'NP']+[x.get_attribute('innerText') for x in x[15].find_elements_by_tag_name ('td')[3:-1]]
        div = [i,cat,'Div']+[x.get_attribute('innerText') for x in x[31].find_elements_by_tag_name ('td')[3:-1]]
        div_pay = [i,cat,'Div Payout']+[x.get_attribute('innerText') for x in x[34].find_elements_by_tag_name ('td')[3:-1]]
#         net_worth = [i,cat,'Net Worth']+[x.get_attribute('innerText') for x in x[39].find_elements_by_tag_name ('td')[3:-1]]
        nosh = [i,cat,'NOSH']+[x.get_attribute('innerText') for x in x[42].find_elements_by_tag_name ('td')[3:-1]]
        np_margin = [i,cat,'NP Margin']+[x.get_attribute('innerText') for x in x[47].find_elements_by_tag_name ('td')[3:-1]]
        roe = [i,cat,'ROE']+[x.get_attribute('innerText') for x in x[50].find_elements_by_tag_name ('td')[3:-1]]
        eps = [i,cat,'EPS']+[x.get_attribute('innerText') for x in x[58].find_elements_by_tag_name ('td')[3:-1]]
        eps_gr = [i,cat,'EPS GR']+[x.get_attribute('innerText') for x in x[59].find_elements_by_tag_name ('td')[3:-1]]
        dps = [i,cat,'DPS']+[x.get_attribute('innerText') for x in x[61].find_elements_by_tag_name ('td')[3:-1]]
        price = [i,cat,'Price']+[x.get_attribute('innerText') for x in x[84].find_elements_by_tag_name ('td')[3:-1]]
        pe = [i,cat,'P/E']+[x.get_attribute('innerText') for x in x[88].find_elements_by_tag_name ('td')[3:-1]]
        dy = [i,cat,'DY']+[x.get_attribute('innerText') for x in x[94].find_elements_by_tag_name ('td')[3:-1]]
        data3 = pd.concat([data3, pd.DataFrame([year_end, net_profit, div, div_pay, nosh, np_margin, roe, eps, eps_gr, dps, price, pe, dy])])
    except:
        error.append(i)
driver.quit()
print('Time taken: ',time()-start_time)
data3.columns = ['code','category','detail','Y1','Y2','Y3','Y4','Y5','Y6','Y7','Y8','Y9','Y10']
data3['code'] = data3['code'].astype(str)
print(data3.shape)
data3.to_csv('klse_investor_10yr.csv',index=False)
data3.head(1)

100%|████████████████████████████████████████████████████████████████████████████████| 170/170 [12:35<00:00,  3.71s/it]


Time taken:  759.9208648204803
(2210, 13)


,code,category,detail,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9,Y10
0,5622,Construction,Year End,31/12/18,31/12/17,31/05/17,31/05/16,31/05/15,31/05/14,31/05/13,31/05/12,31/05/11,31/05/10


In [13]:
data3[data3.code=='2836']

,code,category,detail,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9,Y10


### Phase 3: Process each stock

In [14]:
error4, result4, symbol= [], [], ['  -  ', '- %']
for i in tqdm(data3.code.unique()):
    stock = data3[data3.code == i].fillna('  -  ')
    try:
        for y in [float(x.replace('%','').replace(',','')) for x in stock[stock.detail == 'NP'].values[0][3:].tolist() if x not in symbol]: assert(y >= 0)
        try:
            for y in [float(x.replace('%','').replace(',','')) for x in stock[stock.detail == 'EPS GR'].values[0][3:].tolist() if x not in symbol]: assert(y >= 0)  
            temp = [float(x.replace('%','')) for x in stock[stock.detail == 'ROE'].values[0][3:].tolist() if x not in symbol]
            nY = len(temp)
            try:
                assert(nY >= 5)
                roe_avg = np.around(np.average(temp), 2)
                temp = [float(x.replace('%','')) for x in stock[stock.detail == 'EPS GR'].values[0][3:3+nY].tolist() if x not in symbol]
                eps_gr_avg = np.around(sum(temp)/nY, 2)
                temp = [float(x.replace('%','')) for x in stock[stock.detail == 'NP Margin'].values[0][3:].tolist() if x not in symbol]
                np_margin_avg = np.around(np.average(temp), 2)
                temp = [float(x) for x in stock[stock.detail == 'P/E'].values[0][3:3+nY].tolist() if x not in symbol]
                pe_now = temp[0]
                pe_avg = np.around((max(temp) + min(temp))/2, 2)
                eps_now = float(stock[stock.detail == 'EPS'].values[0][3])
                dps_now = float(stock[stock.detail == 'DPS'].values[0][3])
                eps = [float(x) for x in stock[stock.detail == 'EPS'].values[0][3:3+nY].tolist() if x not in symbol]
                dps = [float(x) for x in stock[stock.detail == 'DPS'].values[0][3:3+nY].tolist() if x not in symbol]
                dpayout = np.around(np.average([dps[i]/eps[i]*100 for i in range(nY)]),2)
                temp = [float(x) for x in stock[stock.detail == 'DY'].values[0][3:3+nY].tolist() if x not in symbol]
                dy = np.around(np.average(temp),2)
                result4.append([i, nY, roe_avg, eps_gr_avg, np_margin_avg, pe_avg, pe_now, eps_now, dps_now, dpayout, dy])
            except:
                error4.append([i,'Less than 5 years'])
        except:
            error4.append([i,'Negavtive EPS Growth Rate'])
    except:
        error4.append([i,'Negative Net Profit'])

100%|███████████████████████████████████████████████████████████████████████████████| 170/170 [00:00<00:00, 329.77it/s]


In [15]:
er_df = pd.DataFrame(error4, columns=['code','reason'])
er_df.reason.value_counts()

Negavtive EPS Growth Rate    94
Negative Net Profit          76
Name: reason, dtype: int64

From the observation above, it is shown that the stocks filtered do not have consitent positve EPS growth rate, which is different from what is taught from the book. Thus, the eps growth rate filter is suspended.

In [16]:
error4, result4, symbol= [], [], ['  -  ', '- %']
for i in tqdm(data3.code.unique()):
    stock = data3[data3.code == i].fillna('  -  ')
    try:
        for y in [float(x.replace('%','').replace(',','')) for x in stock[stock.detail == 'NP'].values[0][3:].tolist() if x not in symbol]: assert(y >= 0)
#         try:
#             for y in [float(x.replace('%','').replace(',','')) for x in stock[stock.detail == 'EPS GR'].values[0][3:].tolist() if x not in symbol]: assert(y >= 0)  
        temp = [float(x.replace('%','')) for x in stock[stock.detail == 'ROE'].values[0][3:].tolist() if x not in symbol]
        nY = len(temp)
        try:
            assert(nY >= 5)
            roe_avg = np.around(np.average(temp), 2)
            temp = [float(x.replace('%','')) for x in stock[stock.detail == 'EPS GR'].values[0][3:3+nY].tolist() if x not in symbol]
            eps_gr_avg = np.around(sum(temp)/nY, 2)
            temp = [float(x.replace('%','')) for x in stock[stock.detail == 'NP Margin'].values[0][3:].tolist() if x not in symbol]
            np_margin_avg = np.around(np.average(temp), 2)
            temp = [float(x) for x in stock[stock.detail == 'P/E'].values[0][3:3+nY].tolist() if x not in symbol]
            pe_now = temp[0]
            pe_avg = np.around((max(temp) + min(temp))/2, 2)
            eps_now = float(stock[stock.detail == 'EPS'].values[0][3])
            dps_now = float(stock[stock.detail == 'DPS'].values[0][3])
            eps = [float(x) for x in stock[stock.detail == 'EPS'].values[0][3:3+nY].tolist() if x not in symbol]
            dps = [float(x) for x in stock[stock.detail == 'DPS'].values[0][3:3+nY].tolist() if x not in symbol]
            dpayout = np.around(np.average([dps[i]/eps[i]*100 for i in range(nY)]),2)
            temp = [float(x) for x in stock[stock.detail == 'DY'].values[0][3:3+nY].tolist() if x not in symbol]
            dy = np.around(np.average(temp),2)
            result4.append([i, nY, roe_avg, eps_gr_avg, np_margin_avg, pe_avg, pe_now, eps_now, dps_now, dpayout, dy])
        except:
            error4.append([i,'Less than 5 years'])
#         except:
#             error4.append([i,'Negavtive EPS Growth Rate'])
    except:
        error4.append([i,'Negative Net Profit'])

100%|███████████████████████████████████████████████████████████████████████████████| 170/170 [00:00<00:00, 190.13it/s]


In [17]:
er_df = pd.DataFrame(error4, columns=['code','reason'])
er_df.reason.value_counts()

Negative Net Profit    76
Less than 5 years      14
Name: reason, dtype: int64

In [18]:
data5 = pd.DataFrame(result4, columns=['code','num_year','ROE avg','EPS GR avg','NPM avg','P/E avg','P/E','EPS','DPS','DPayout avg','DY avg'])
data5['code'] = data5['code'].astype(str)
data5 = data5[data5['ROE avg'] >= 5]
data5 = data5[data5['EPS GR avg'] >= 5]
data5 = data5[data5['P/E'] < data5['P/E avg']]
data5 = data5.merge(data2,'left')
data5 = data5.merge(data1[['code','ROI','price','category','name']],'left')
print(data5.shape)
data5.sort_values('ROI',ascending=False,inplace=True)
data5.reset_index(drop=True,inplace=True)
data5.to_csv('klse_investor_stock_phase3.csv',index=False)
data5.head(1)

(47, 17)


,code,num_year,ROE avg,EPS GR avg,NPM avg,P/E avg,P/E,EPS,DPS,DPayout avg,DY avg,52w low,52w high,ROI,price,category,name
0,8591,10,7.6,38.61,7.86,9.62,2.1,41.4,4.5,31.22,5.14,0.79,1.1,47.61,0.87,Construction,CREST BUILDER HOLDINGS BERHAD


In [19]:
# This means that Carlsberg stock did not pass our filter
data5[data5.code=='2836']

,code,num_year,ROE avg,EPS GR avg,NPM avg,P/E avg,P/E,EPS,DPS,DPayout avg,DY avg,52w low,52w high,ROI,price,category,name


### Phase 4: Intrinsic Value & Fair Value

In [3]:
result5 = []
for n, x in tqdm(data5.iterrows(), total=len(data5)):
    price_10y = [np.around(x['price'] * ((1 + (x['EPS GR avg'] / 100))**i) ,2) for i in range(5)]
    total_eps = np.around(sum(price_10y), 2)
    expected_price = np.around(price_10y[-1] * x['P/E avg'],2)
    total_dvd = np.around(total_eps * (x['DPayout avg'] / 100), 2)
    total_return = np.around(total_eps + total_dvd, 2)
    intrinsic = np.around(total_return / ((1 + (x['EPS GR avg'] / 100))**5), 2)
    price_A = np.around(intrinsic * 0.75, 2)
    price_B = np.around((x['52w high'] - x['52w low']) * 0.33 + x['52w low'], 2)
    result5.append([str(x['code']), intrinsic, price_A, price_B])

100%|████████████████████████████████████████████████████████████████████████████████| 47/47 [00:00<00:00, 1504.32it/s]


In [4]:
data6 = data5.merge(pd.DataFrame(result5, columns=['code','intrinsic','price_A','price_B']))
print(data6.shape)
data6.to_csv('klse_investor_intrinsic.csv',index=False)
data6.head(1)

(47, 20)


,code,num_year,ROE avg,EPS GR avg,NPM avg,P/E avg,P/E,EPS,DPS,DPayout avg,DY avg,52w low,52w high,ROI,price,category,name,intrinsic,price_A,price_B
0,8591,10,7.6,38.61,7.86,9.62,2.1,41.4,4.5,31.22,5.14,0.79,1.1,47.61,0.87,Construction,CREST BUILDER HOLDINGS BERHAD,2.38,1.78,0.89


In [22]:
# Empty because it did not pass the previous filter, you can try with the other 24 stocks
data6[data6.code=='2836']

,code,num_year,ROE avg,EPS GR avg,NPM avg,P/E avg,P/E,EPS,DPS,DPayout avg,DY avg,52w low,52w high,ROI,price,category,name,intrinsic,price_A,price_B


In [23]:
# Stocks which have prices lower than fair value price B
data6[data6.price < data6.price_B]

,code,num_year,ROE avg,EPS GR avg,NPM avg,P/E avg,P/E,EPS,DPS,DPayout avg,DY avg,52w low,52w high,ROI,price,category,name,intrinsic,price_A,price_B
0,8591,10,7.60,38.61,7.86,9.62,2.10,41.40,4.50,31.22,5.14,0.790,1.100,47.61,0.870,Construction,CREST BUILDER HOLDINGS BERHAD,2.38,1.78,0.89
4,7323,10,12.23,11.18,34.08,5.80,4.70,13.42,1.50,21.61,3.85,0.555,0.940,21.30,0.630,Property,KEN HOLDINGS BERHAD,2.82,2.11,0.68
5,9296,10,13.58,133.67,28.42,53.08,4.74,26.03,7.00,134.62,8.90,1.060,1.720,21.08,1.230,Finance,RCE CAPITAL BERHAD,2.13,1.60,1.28
9,7029,10,6.07,23.40,4.60,11.15,5.46,10.54,1.50,10.68,1.48,0.545,0.730,18.32,0.575,Industrial Products,MASTER-PACK GROUP BERHAD,1.77,1.33,0.61
10,5127,10,9.04,34.70,92.00,14.04,5.79,14.17,6.10,77.60,7.45,0.810,0.900,17.29,0.820,REITS,AMANAHRAYA REAL ESTATE INVESTMENT TRUST,3.25,2.44,0.84
12,4944,10,5.24,15.12,1.10,9.61,6.20,10.05,2.00,31.72,2.93,0.580,0.825,16.14,0.630,Industrial Products,NYLEX (MALAYSIA) BERHAD,2.77,2.08,0.66
14,7034,10,9.64,15.83,4.77,8.30,6.61,33.29,8.00,27.13,3.96,2.070,3.080,15.13,2.200,Industrial Products,THONG GUAN INDUSTRIES BERHAD,9.19,6.89,2.40
17,5065,10,6.16,12.82,3.14,8.31,7.02,12.67,2.50,8.65,1.03,0.880,1.780,14.24,0.890,Industrial Products,ORNAPAPER BERHAD,3.41,2.56,1.18
18,3883,10,5.30,32.21,3.82,15.05,7.04,25.42,4.50,17.74,3.07,1.500,2.950,14.20,1.790,Industrial Products,MUDA HOLDINGS BERHAD,4.93,3.70,1.98
21,7091,10,8.12,5.62,8.87,10.24,7.59,14.12,3.80,21.11,1.92,0.900,1.100,13.17,0.950,Industrial Products,UNIMECH GROUP BERHAD,4.89,3.67,0.97


In [24]:
# Stocks which have prices lower than fair value price B and average NPM of 10% or higher
data6[(data6.price < data6.price_B) & (data6['NPM avg'] >= 10)]

,code,num_year,ROE avg,EPS GR avg,NPM avg,P/E avg,P/E,EPS,DPS,DPayout avg,DY avg,52w low,52w high,ROI,price,category,name,intrinsic,price_A,price_B
4,7323,10,12.23,11.18,34.08,5.80,4.70,13.42,1.50,21.61,3.85,0.555,0.94,21.30,0.63,Property,KEN HOLDINGS BERHAD,2.82,2.11,0.68
5,9296,10,13.58,133.67,28.42,53.08,4.74,26.03,7.00,134.62,8.90,1.060,1.72,21.08,1.23,Finance,RCE CAPITAL BERHAD,2.13,1.60,1.28
10,5127,10,9.04,34.70,92.00,14.04,5.79,14.17,6.10,77.60,7.45,0.810,0.90,17.29,0.82,REITS,AMANAHRAYA REAL ESTATE INVESTMENT TRUST,3.25,2.44,0.84
27,5152,10,13.94,9.24,14.58,10.31,7.75,13.03,3.00,42.93,4.83,0.905,1.25,12.90,1.01,Industrial Products,MUAR BAN LEE GROUP BERHAD,5.59,4.19,1.02
40,1171,10,18.88,19.74,18.99,15.86,8.91,10.32,0.00,37.21,4.03,0.890,1.25,11.22,0.92,Finance,MALAYSIA BUILDING SOCIETY BERHAD,3.79,2.84,1.01
42,3514,10,10.92,8.58,10.18,11.32,8.97,1.34,0.30,41.27,4.98,0.115,0.15,11.15,0.12,Consumer,MARCO HOLDINGS BERHAD,0.66,0.50,0.13
43,1066,10,12.90,7.95,19.87,11.04,9.20,57.49,20.50,33.24,2.87,4.990,5.94,10.87,5.29,Finance,RHB BANK BERHAD,28.17,21.13,5.30
45,5110,10,9.29,8.03,72.94,12.04,9.63,13.61,9.13,77.51,7.04,1.290,1.52,10.39,1.31,REITS,UOA REAL ESTATE INVESTMENT TRUST,9.28,6.96,1.37
46,1023,10,13.28,5.97,28.02,13.59,9.73,59.67,25.00,44.31,3.21,5.210,7.36,10.28,5.71,Finance,CIMB GROUP HOLDINGS BERHAD,34.73,26.05,5.92


In practical, intrinsic value and price A are way beyond from the current price because there is no consistency of positive growth rate for any stock, my suggestion will be excluding the idea of positive growth rate, but filter stocks with positive earnings or net profit margin at 5% or 10%  
 <br />
Another method would be filtering stocks according to the dividend yield over the past 10 years. Although this method is not recommended by the author,but it returns more stock options and most of them has dividend yield of 10% or higher

### Process by Dividend Yield

In [25]:
data7 = pd.DataFrame(result4, columns=['code','num_year','ROE avg','EPS GR avg','NPM avg','P/E avg','P/E','EPS','DPS','DPayout avg','DY avg'])
data7['code'] = data7['code'].astype(str)
# The commented code below are some filters which can be tuned
# data7 = data7[data7['ROE avg'] >= 5]
# data7 = data7[data7['EPS GR avg'] >= 5]
# data7 = data7[data7['P/E'] < data5['P/E avg']]
data7 = data7.merge(data2,'left')
data7 = data7.merge(data1[['code','ROI','price','category','name']],'left')
print(data7.shape)
data7.sort_values('ROI',ascending=False,inplace=True)
data7.reset_index(drop=True,inplace=True)
data7.head(1)

(80, 17)


,code,num_year,ROE avg,EPS GR avg,NPM avg,P/E avg,P/E,EPS,DPS,DPayout avg,DY avg,52w low,52w high,ROI,price,category,name
0,8591,10,7.6,38.61,7.86,9.62,2.1,41.4,4.5,31.22,5.14,0.79,1.1,47.61,0.87,Construction,CREST BUILDER HOLDINGS BERHAD


In [26]:
data7.sort_values('DY avg',ascending=False)

,code,num_year,ROE avg,EPS GR avg,NPM avg,P/E avg,P/E,EPS,DPS,DPayout avg,DY avg,52w low,52w high,ROI,price,category,name
70,1724,10,10.67,8.95,15.98,5.88,9.20,22.20,8.50,49.24,11.92,1.770,2.260,10.87,2.040,Property,PARAMOUNT CORPORATION BERHAD
13,9571,10,14.55,-6.02,10.67,5.10,5.60,4.98,1.50,33.57,9.31,0.225,0.870,17.85,0.280,Construction,MITRAJAYA HOLDINGS BERHAD
7,9296,10,13.58,133.67,28.42,53.08,4.74,26.03,7.00,134.62,8.90,1.060,1.720,21.08,1.230,Finance,RCE CAPITAL BERHAD
54,9369,10,9.97,-2.13,6.57,7.88,8.35,11.61,5.00,54.56,7.84,0.880,1.100,11.97,0.970,Consumer,TEO GUAN LEE CORPORATION BERHAD
15,5127,10,9.04,34.70,92.00,14.04,5.79,14.17,6.10,77.60,7.45,0.810,0.900,17.29,0.820,REITS,AMANAHRAYA REAL ESTATE INVESTMENT TRUST
77,5110,10,9.29,8.03,72.94,12.04,9.63,13.61,9.13,77.51,7.04,1.290,1.520,10.39,1.310,REITS,UOA REAL ESTATE INVESTMENT TRUST
48,7089,10,17.96,8.03,7.64,6.31,8.10,32.10,11.50,38.80,6.94,2.250,3.310,12.35,2.600,Consumer,LII HEN INDUSTRIES BHD.
63,5100,10,9.74,9.67,5.93,11.14,8.81,11.23,6.00,65.98,6.06,0.850,1.170,11.35,0.990,Industrial Products,BP PLASTICS HOLDING BHD
27,5162,9,13.80,-5.84,1.91,5.98,7.02,13.70,5.00,37.49,6.04,0.910,1.220,14.25,0.960,Technology,VSTECS BERHAD
17,5191,9,34.44,-4.94,29.76,3.82,5.91,12.78,6.70,33.36,5.89,0.710,1.050,16.92,0.755,Property,TAMBUN INDAH LAND BERHAD


 #### Thank you for reading, using, and exploring this, hope to see you soon